First Version
Abhijeet

In [1]:
import pandas as pd
import numpy as np
from pandas.core.series import Series
import glob
import datetime

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [26]:
import mysql.connector

In [27]:
config = {
  'user': 'jlevin',
  'password': 'ucsdcapstone',
  'host': 'mandrake.arvixe.com',
  'database': 'heavylifters',
  'raise_on_warnings': True,
}

conn = mysql.connector.connect(**config)

#conn.close()

In [5]:
#cursor = conn.cursor()

#query = ("SELECT * FROM heavylifters.raw_data_product_2011 LIMIT 20;")
#query = ("SELECT * FROM heavylifters.data_panel_08_to_11 LIMIT 20;")
query = ("SELECT PANID, MINUTE_KEY, UNITS, dp.OUTLET, DOLLARS, dp.IRI_KEY, COLUPC , dp.DATE_YEAR, dp.CATEGORY , \
          WEEK_KEY, left(COLUPC,2)/1 as SY, MID(colupc,3,2) as GE, MID(colupc,5,5) as VEND, \
          MID(colupc,9,5)/1 as ITEM, MINUTE_KEY/1440, MARKET_NAME, DESCRIPTION \
          FROM heavylifters.data_panel_08_to_11 as dp \
          join data_delivery_stores_01_to_11 as ds \
          on dp.IRI_KEY = ds.IRI_KEY \
          join dim_category as dc \
          on dp.CATEGORY = dc.CATEGORY\
          and dp.Date_Year = ds.Date_Year \
          where dp.Date_Year = 2011 ORDER by Market_Name LIMIT 5000;")




In [6]:
df_mysql = pd.read_sql(query, con=conn)    
print 'loaded dataframe from MySQL. records:', len(df_mysql)

loaded dataframe from MySQL. records: 5000


In [30]:
#cursor = conn.cursor()

#query = ("SELECT * FROM heavylifters.raw_data_product_2011 LIMIT 20;")
#query = ("SELECT * FROM heavylifters.data_panel_08_to_11 LIMIT 20;")
query2 = ("SELECT distinct sp11.IRI_Key,sp11.OUTLET,sp11.WEEK_KEY,ds.Market_Name, \
ds.Date_Year,ds.MskdName,ds.OPEN_WEEK,ds.CLSD_WEEK,ds.EST_ACV,ds.Date_Year, \
sp11.CATEGORY,sp11.OUTLET,sp11.DOLLARS,sp11.UNITS,sp11.Count_Rows \
FROM data_delivery_stores_01_to_11 as ds, summary_product_2011 as sp11 \
WHERE ds.Market_Name='LOS ANGELES' OR ds.Market_Name='CHICAGO' \
and ds.IRI_KEY = sp11.IRI_Key LIMIT 1000;")




In [31]:
df_mysqls2 = pd.read_sql(query2, con=conn)    
print 'loaded dataframe from MySQL. records:', len(df_mysqls2)

loaded dataframe from MySQL. records: 5000


In [33]:
#cursor.close()
conn.close()

In [35]:
df_mysqls2.columns,df_mysqls2.head(20)

(Index([u'IRI_Key', u'OUTLET', u'WEEK_KEY', u'Market_Name', u'Date_Year',
        u'MskdName', u'OPEN_WEEK', u'CLSD_WEEK', u'EST_ACV', u'Date_Year',
        u'CATEGORY', u'OUTLET', u'DOLLARS', u'UNITS', u'Count_Rows'],
       dtype='object'),
     IRI_Key OUTLET  WEEK_KEY  Market_Name  Date_Year    MskdName  OPEN_WEEK  \
 0    200272     GK      1635  LOS ANGELES       2011  Chain104\r        873   
 1    200272     GK      1635  LOS ANGELES       2011  Chain104\r        473   
 2    200272     GK      1635  LOS ANGELES       2011  Chain104\r        873   
 3    200272     GK      1635  LOS ANGELES       2011  Chain94 \r       1092   
 4    200272     GK      1635  LOS ANGELES       2011  Chain94 \r       1093   
 5    200272     GK      1635  LOS ANGELES       2011  Chain94 \r       1092   
 6    200272     GK      1635  LOS ANGELES       2011  Chain94 \r       1092   
 7    200272     GK      1635  LOS ANGELES       2011  Chain94 \r       1092   
 8    200272     GK      1635  LOS AN

In [25]:
for i in df_mysql.columns:
    x=len(df_mysql[i].unique())
    print i,x
    if(x<7):
        print type(df_mysql[i][0]),df_mysql[i].unique()

PANID 684
MINUTE_KEY 2678
UNITS 11
OUTLET 2
<type 'unicode'> [u'G' u'D']
DOLLARS 413
IRI_KEY 2
<type 'numpy.int64'> [257871 651444]
COLUPC 540
DATE_YEAR 1
<type 'numpy.int64'> [2011]
CATEGORY 3
<type 'unicode'> [u'BL' u'BR' u'CB']
WEEK_KEY 52
SY 6
<type 'numpy.float64'> [  0.  88.   8.   3.   6.   7.]
GE 29
VEND 153
ITEM 523
MINUTE_KEY/1440 2678
MARKET_NAME 1
<type 'unicode'> [u'EAU CLAIRE']
DESCRIPTION 3
<type 'unicode'> [u'Blades' u'Beer' u'Carbonated Beverages']


In [17]:
df_mysql.shape,len(df_mysql['DESCRIPTION'].unique()),df_mysql.columns,df_mysql.head(20)

((5000, 17),
 3,
 Index([u'PANID', u'MINUTE_KEY', u'UNITS', u'OUTLET', u'DOLLARS', u'IRI_KEY',
        u'COLUPC', u'DATE_YEAR', u'CATEGORY', u'WEEK_KEY', u'SY', u'GE',
        u'VEND', u'ITEM', u'MINUTE_KEY/1440', u'MARKET_NAME', u'DESCRIPTION'],
       dtype='object'),
       PANID  MINUTE_KEY  UNITS OUTLET  DOLLARS  IRI_KEY          COLUPC  \
 0   3104844        2108      1      G     0.99   257871  0012429129059    
 1   3169318        7765      1      G     0.99   257871  0012429129059    
 2   3377705        6570      1      G     0.99   257871  0012429129059    
 3   3129692        6654      1      G     4.19   257871  8849999874528    
 4   3343079        5529      1      G     4.19   257871  8849999874528    
 5   3154823        9741      1      G    17.49   257871  0014740007400    
 6   3109108         458      1      G    17.39   257871  0014740007404    
 7   3109108        1963      1      G     7.89   257871  0014740007811    
 8   3136408         499      1      G     7.

In [13]:
def convert_week(x):
    start_epoch_arr= x.values.astype(int)
    end_epoch_arr= np.array(start_epoch_arr)
    
    start_epoch_arr= (start_epoch_arr-400)*7+31900-6
    start_epoch_arr = (start_epoch_arr-25569) * 86400
    
    end_epoch_arr = (end_epoch_arr-400)*7+31900
    end_epoch_arr= (end_epoch_arr-25569) * 86400
    
    return (start_epoch_arr, end_epoch_arr)

def get_transaction_time(x, start_epoch_arr):
    transaction_time_epoch_arr= x.values.astype(int)
    transaction_time_epoch_arr= start_epoch_arr + (transaction_time_epoch_arr * 60)
    return transaction_time_epoch_arr

def get_datetime(transaction_time_epoch_arr):
    t_shape= np.shape(transaction_time_epoch_arr)
    month_arr= np.zeros(t_shape, dtype=int)
    day_arr=np.zeros(t_shape, dtype=int)
    year_arr= np.zeros(t_shape, dtype=int)
    hour_arr= np.zeros(t_shape, dtype=int)
    minute_arr= np.zeros(t_shape, dtype=int)
    dayofweek_arr= np.zeros(t_shape, dtype=int)
    week_num_arr= np.zeros(t_shape, dtype=int)
    
    for i in xrange(t_shape[0]):
        t_time= transaction_time_epoch_arr[i]
        dt= datetime.datetime.utcfromtimestamp(t_time)
        month_arr[i]= dt.month
        day_arr[i]= dt.day
        year_arr[i]= dt.year
        hour_arr[i]= dt.hour
        minute_arr[i]= dt.minute
        dayofweek_arr[i]= dt.weekday()
        week_num_arr[i]= dt.isocalendar()[1]
    
    return (month_arr, day_arr, year_arr, hour_arr, minute_arr, dayofweek_arr, week_num_arr)

In [14]:
def parse_colupc(colupc_arr):
    t_shape= np.shape(colupc_arr)
    sy_arr= np.zeros(t_shape, dtype=int)
    ge_arr=np.zeros(t_shape, dtype=int)
    vend_arr= np.zeros(t_shape, dtype=int)
    item_arr= np.zeros(t_shape, dtype=int)
    for i, colupc in enumerate(colupc_arr):
        assert len(colupc) == 13
        # sy, gen, vend, item
        m_as= [colupc[:2], colupc[2:3], colupc[3:8], colupc[8:]]
        m_outs= [sy_arr, ge_arr, vend_arr, item_arr]
        for j in xrange(4):
            a= m_as[j]
            out= m_outs[j]
            out[i]= int(a)
    return (sy_arr, ge_arr, vend_arr, item_arr)

In [15]:
def parse_df(my_df):
    m_df = my_df
    
    s_week_df= m_df["WEEK_KEY"]
    start_time_epoch_arr, end_time_epoch_arr= convert_week(s_week_df)
    r1= pd.DataFrame({
        'START_TIME_EPOCH_S' : start_time_epoch_arr
    })
    r2= pd.DataFrame({
        'END_TIME_EPOCH_S' : end_time_epoch_arr
    })
    m_df= pd.concat([m_df, r1, r2], axis=1)
    #
    s_minute_df= m_df["MINUTE_KEY"]
    transaction_time_epoch_arr= get_transaction_time(s_minute_df, start_time_epoch_arr)
    t_result= pd.DataFrame({
        'TRANSACTION_TIME_EPOCH_S' : transaction_time_epoch_arr
    })
    # extract month, day, year, hour, minute
    month_arr, day_arr, year_arr, hour_arr, minute_arr, dayofweek_arr, week_num_arr= get_datetime(transaction_time_epoch_arr)
    r1= pd.DataFrame({
        'MONTH' : month_arr
    })
    r2= pd.DataFrame({
        'DAY' : day_arr
    })
    r3= pd.DataFrame({
        'YEAR' : year_arr
    })
    r4= pd.DataFrame({
        'HOUR_OF_DAY' : hour_arr
    })
    r5= pd.DataFrame({
        'MINUTE' : minute_arr
    })
    r6= pd.DataFrame({
        'DAYOFWEEK' : dayofweek_arr
    })
    r7= pd.DataFrame({
        'WEEKNUM' : week_num_arr
    })
    m_df.drop("MINUTE_KEY", axis=1, inplace=True)
    #m_df= pd.concat([m_df, t_result, r1, r2, r3, r4, r5, r6, r7], axis=1)
    m_df= pd.concat([m_df, t_result, r1, r2, r3, r4, r5, r6, r7], axis=1)
    
    # Parse COLUPC to SY GE VEND ITEM
    # 18417375121 ->   
    #  item: 75121
    #  vender: 84173
    #  generation: 1
    #  system: 00 (zero padded)
    #s_colupc_df= m_df["COLUPC"]
    #sy_arr, ge_arr, vend_arr, item_arr = parse_colupc(s_colupc_df)
    #r1= pd.DataFrame({
    #    'SY' : sy_arr
    #})
    #r2= pd.DataFrame({
    #    'GE' : ge_arr
    #})
    #r3= pd.DataFrame({
    #    'VEND' : vend_arr
    #})
    #r4= pd.DataFrame({
    #    'ITEM' : item_arr
    #})
    #m_df= pd.concat([m_df, r1, r2, r3, r4], axis=1)
    
    return m_df

In [16]:
new_df = parse_df(df_mysql)

In [17]:
new_df.shape

(30000, 26)

In [18]:
new_df.head()

PANID  UNITS OUTLET  DOLLARS  IRI_KEY          COLUPC  DATE_YEAR  \
0  3104844      1      G     0.99   257871  0012429129059        2011   
1  3169318      1      G     0.99   257871  0012429129059        2011   
2  3377705      1      G     0.99   257871  0012429129059        2011   
3  3129692      1      G     4.19   257871  8849999874528        2011   
4  3343079      1      G     4.19   257871  8849999874528        2011   

  CATEGORY  WEEK_KEY  SY   ...    START_TIME_EPOCH_S END_TIME_EPOCH_S  \
0       BL      1653   0   ...            1304294400       1304812800   
1       BL      1646   0   ...            1300060800       1300579200   
2       BL      1656   0   ...            1306108800       1306627200   
3       BL      1678  88   ...            1319414400       1319932800   
4       BL      1651  88   ...            1303084800       1303603200   

   TRANSACTION_TIME_EPOCH_S  MONTH DAY  YEAR  HOUR_OF_DAY  MINUTE  DAYOFWEEK  \
0                1304420880      5   3  2011           11       8          1   
1                1300526700      3  19  2011            9      25          5   
2                1306503000      5  27  2011           13      30          4   
3                1319813640     10  28  2011           14      54          4   
4                1303416540      4  21  2011           20       9          3   

   WEEKNUM  
0       18  
1       11  
2       21  
3       43  
4       16  

[5 rows x 26 columns]

In [19]:
new_df.columns

Index([                   u'PANID',                    u'UNITS',
                         u'OUTLET',                  u'DOLLARS',
                        u'IRI_KEY',                   u'COLUPC',
                      u'DATE_YEAR',                 u'CATEGORY',
                       u'WEEK_KEY',                       u'SY',
                             u'GE',                     u'VEND',
                           u'ITEM',          u'MINUTE_KEY/1440',
                    u'MARKET_NAME',              u'DESCRIPTION',
             u'START_TIME_EPOCH_S',         u'END_TIME_EPOCH_S',
       u'TRANSACTION_TIME_EPOCH_S',                    u'MONTH',
                            u'DAY',                     u'YEAR',
                    u'HOUR_OF_DAY',                   u'MINUTE',
                      u'DAYOFWEEK',                  u'WEEKNUM'],
      dtype='object')

In [20]:
t_df = new_df.groupby(['DAY', 'DESCRIPTION'])['DOLLARS'].sum()

In [23]:
t_df

DAY  DESCRIPTION         
1    Beer                     712.7100
     Blades                     8.7900
     Carbonated Beverages    2133.6261
     Cigarettes               530.9200
     Cold Cereal              134.5400
     Deodorant                 25.5500
     Facial Tissue             74.1900
     Frozen Dinner            631.4700
     Frozen Pizza             540.6500
     Hotdog                   174.4900
     Household Cleaners        38.4100
     Laundry Detergent        124.5100
     Margarine Butter          30.8600
2    Beer                     942.8100
     Blades                     7.0800
     Carbonated Beverages    1831.5500
     Cigarettes               625.9000
     Cold Cereal              154.5500
     Deodorant                  8.5700
     Facial Tissue             87.2600
     Frozen Dinner            681.0700
     Frozen Pizza             607.1100
     Hotdog                   187.6100
     Household Cleaners        30.0100
     Laundry Detergent        203.8100